# Python/Jupyter noptebook pdf merging
### I had acces to a couple thousand pdf files in directories that need to be processed,
#### here's my implementation

import os, check if main directory path exists

In [1]:
import os

In [2]:
file_root_dir = "D:/E_filing/oct3_23"
print(os.path.exists(file_root_dir))

True


### File structure
The main directory contains ~3k sub-dicrectories  
these direcories contain scanned .pdf documents(could contain more than one pdf file). the direcory names are unique ten digit numbers identifying customers
- 1234567890  
         scan.pdf
-  1123456789  
         scan.pdf
-  1223456789  
         scan.pdf  
   ...  
### the problem:
during scanning there were a few issues:
- scanner couldnt finish scanning whole document
- document was too large to scan in one go
- power outtages
so a lot of directories contained multiple pdfs and some directories contained other directories,  
multiple directories contained the same customers documents
### To do list  
1. fix nested directories
2. merge multiple pdfs in each directory
3. fix direcory name sizes
4. move around files so that each direcory contains all document of a single customer
5. finally rename the pdfs with the name of thier root directories(the unique ten digit id)

### 1. Used this bit to find nested directories(there were just a bit over a dozen so i fixed them in explorer)  

In [3]:
dircount = 0
dir_list = []
dir_abnormal = []
for root, dirs, files in os.walk(file_root_dir, topdown=True):
    # skip root directory
    if root == "D:/E_filing/oct3_23":
        continue
    # name will eist if theres a dirs contains sub directories
    for name in dirs:
        dircount += 1
        dir_list.append(name)
        dir_abnormal.append(dirs)
        print(root)
print(dir_abnormal)
print("nested directories = {}".format(dircount))


[]
nested directories = 0


### 3. making sure all directories have a ten digit number as names(doesnt include pdfs of a single id in ,multiple directories)  
the ids for these directories neede to be checked aginst the pdf content so it's a manual job :|  

In [5]:
import re
import shutil
dirs = []
abdir = 0
entries = os.scandir(file_root_dir)
dest = "D:/E_filing/ab/"
back_dest = "D:/E_filing/oct3_23/"
for entry in entries:
    name = entry.name
    if len(name) != 10 and not re.search(r'.\(.\)$', name):
        # move files away because editing with file explorer created issues
        #if os.path.exists(entry.path):
        #    shutil.move(entry.path, dest)
        abdir += 1
        dirs.append(entry.name)
print(abdir)
print(dirs)
# check again in destination folder
entries = os.scandir(dest)
abdir = 0
dirs_new = []
for entry in entries:
    name = entry.name
    if len(name) != 10 and not re.search(r'.\(.\)$', name):
        abdir += 1
    else:
        shutil.move(entry.path, back_dest + "/" + entry.name)
        dirs_new.append(entry.path)
print(abdir)
for item in dirs_new:
    print(item)



0
[]
0


4. move around files so that each direcory contains all documents of a single customer

In [6]:
import re
import shutil
dirs = []
abdir = 0
dd = 0
moved = []
entries = os.scandir(file_root_dir)
dest = "D:/E_filing/ab/"
back_dest = "D:/E_filing/oct3_23/"
for entry in entries:
    name = entry.name
    if len(name) != 10 and re.search(r'\(.\)$', name):
        abdir += 1
        if name[:10] not in dirs:
            dirs.append(name[:10])
print("number of multiple directories=", abdir)
print("number of unique id's=", len(dirs))
print(dirs)
entriess = os.scandir(file_root_dir)
for entry in entriess:
    names = entry.name
    if names[:10] in dirs:
        dd += 1
        moved.append(names[:10])
        shutil.move(entry.path, dest + "/" + names)
print(dd)
print(moved)

number of multiple directories= 104
number of unique id's= 54
['0000546960', '0001127134', '0002208885', '0002797415', '0003883372', '0006573774', '0007682928', '0013221392', '0028087451', '0030260750', '0036802307', '0037830376', '0043763841', '0046198146', '0047276300', '0048137582', '0050464470', '0052177400', '0053195797', '0054171726', '0054639186', '0054721163', '0054995808', '0055082157', '0055086548', '0055132232', '0055441170', '0055602183', '0055607011', '0055664797', '0055671667', '0055899198', '0056004351', '0056722600', '0057006775', '0057096769', '0057624952', '0057709068', '0057723382', '0057723711', '0057982929', '0058094975', '0058276245', '0058277731', '0059234640', '0059756473', '0062310065', '0064932957', '0066336398', '0066488313', '0067817420', '0068508574', '0068777768', '0069177841']
119
['0000546960', '0000546960', '0000546960', '0001127134', '0001127134', '0001127134', '0002208885', '0002208885', '0002208885', '0002208885', '0002208885', '0002797415', '0002797

In [7]:
# checking if there are any files left of the previous operation
import re
import shutil
id_list = []
abdir = 0
dd = 0
moved = []

dest = "D:/E_filing/ab/"
back_dest = "D:/E_filing/oct3_23/"

entries_root = os.scandir(file_root_dir)
# check if there are any residual files in root directory that have not been moved
entries_ab = os.scandir(dest)
for id in entries_ab:
    if id.name not in id_list:
        id_list.append(id.name)
dirs = []
for entry in entries_root:
    name = entry.name
    if name[:10] in id_list:
        abdir += 1
        dirs.append(name[:10])
print("number of residual directories=", abdir)
print(dirs)

number of residual directories= 0
[]


### ^success!!  
now move all documents belonging to a single user id to one directory

In [14]:
# dest = "D:/E_filing/ab/"
# id_list contains all directories to be fixed

fixed_list = []
for id in id_list:
    if len(id) == 10:
        fixed_list.append(id)

    

In [18]:
for i in fixed_list:
    print(i)
print("fixed list", len(fixed_list))
print("id_list", len(id_list))

0030260750
0055441170
0055671667
0056722600
0057006775
0057096769
0057709068
0057723382
0058276245
0058277731
0062310065
0066336398
0066488313
0067817420
0068508574
fixed list 15
id_list 119


#### im going to move all other pdfs in the directories that dont have numbers
#### EX move pdfs from '123 (1)' dir to '123' dir
#### the directories without names are in "fixed_list"

In [55]:
entries_ab = os.scandir(dest)
for entry in entries_ab:
    tin = entry.name
    # check if tin has unique relative(dir without "(x) suffix") and iself contains "(x)"
    if tin[:10] in fixed_list and re.search(r'\(.\)$', tin):
        # now scan the dir
        print("in directory ", entry.path)
        files = os.scandir(entry.path)
        # for every pdf in entry
        for file in files:
            if file.is_file():
                # rename file so that moving will not have collisions
                #new_file_name = file.path[:-4] + tin[-2:-1] + file.name[-4:]
                #print("old file name |{}| new file name|{}|".format(file.path, new_file_name))
                #os.rename(file.path, new_file_name)
                print("file name |{}|".format(file.path))
                # get destination
                destination = entry.path[:25] + '/'
                print("moving file |{}| to destination |{}|".format(file.path, destination))
                # !!! move file !!!
                shutil.move(file.path, destination)
                # move to unique relative directory and remane it so it wont have conflicting name
        
    # cleanup, no that the directory is empty remove it
    #os.rmdir(entry.path)

in directory  D:/E_filing/ab/0030260750(2)
in directory  D:/E_filing/ab/0055441170 (2)
in directory  D:/E_filing/ab/0055671667 (1)
in directory  D:/E_filing/ab/0056722600 (1)
in directory  D:/E_filing/ab/0057006775(1)
in directory  D:/E_filing/ab/0057006775(2)
in directory  D:/E_filing/ab/0057096769 (1)
in directory  D:/E_filing/ab/0057709068(1)
in directory  D:/E_filing/ab/0057723382 (1)
in directory  D:/E_filing/ab/0058276245(1)
in directory  D:/E_filing/ab/0058277731(1)
in directory  D:/E_filing/ab/0062310065(1)
in directory  D:/E_filing/ab/0066336398(1)
in directory  D:/E_filing/ab/0066488313(2)
in directory  D:/E_filing/ab/0067817420 (2)
in directory  D:/E_filing/ab/0068508574 (1)


In [59]:
# remove empty directories
entries_ab = os.scandir(dest)

for entry in entries_ab:
    # delete directory
    try:
        os.rmdir(entry.path)
        print("deleted dir {}".format(entry.path))
    except OSError as e:
        pass

deleted dir D:/E_filing/ab/0055082157(1)
deleted dir D:/E_filing/ab/0055441170 (2)
deleted dir D:/E_filing/ab/0055671667 (1)
deleted dir D:/E_filing/ab/0056722600 (1)
deleted dir D:/E_filing/ab/0057006775(1)
deleted dir D:/E_filing/ab/0057006775(2)
deleted dir D:/E_filing/ab/0057096769 (1)
deleted dir D:/E_filing/ab/0057709068(1)
deleted dir D:/E_filing/ab/0057723382 (1)
deleted dir D:/E_filing/ab/0058276245(1)
deleted dir D:/E_filing/ab/0058277731(1)
deleted dir D:/E_filing/ab/0062310065(1)
deleted dir D:/E_filing/ab/0066336398(1)
deleted dir D:/E_filing/ab/0066488313(2)
deleted dir D:/E_filing/ab/0067817420 (2)
deleted dir D:/E_filing/ab/0068508574 (1)


In [63]:
# rename 123(1) dirs to just 123 to make them a unique directory
entries_ab = os.scandir(dest)
id_list = []
for entry in entries_ab:
    tin = entry.name
    id_list.append(entry.name)
    if re.search(r'\(1\)$', tin):
        print("to rename ", entry.path)
        os.rename(entry.path, entry.path[:25])
        print("renamed to ", entry.path[:25])

In [72]:
# update id list and fixed_list
print("total directories", len(id_list))
fixed_list = []
for i in id_list:
    if len(i) == 10:
        fixed_list.append(i)
print("unique list ", len(fixed_list))

total directories 102
unique list  52


#### second pass

In [76]:
entries_ab = os.scandir(dest)
for entry in entries_ab:
    tin = entry.name
    # check if tin has unique relative(dir without "(x) suffix") and iself contains "(x)"
    if tin[:10] in fixed_list and re.search(r'\(.\)$', tin):
        # now scan the dir
        print("in directory ", entry.path)
        files = os.scandir(entry.path)
        # for every pdf in entry
        for file in files:
            if file.is_file():
                # rename file so that moving will not have collisions
                #new_file_name = file.path[:-4] + tin[-2:-1] + file.name[-4:]
                #print("old file name |{}| new file name|{}|".format(file.path, new_file_name))
                #os.rename(file.path, new_file_name)
                #print("file name |{}|".format(file.path))
                # get destination
                destination = entry.path[:25] + '/'
                print("moving file |{}| to destination |{}|".format(file.path, destination))
                # !!! move file !!!
                shutil.move(file.path, destination)

in directory  D:/E_filing/ab/0000546960 (2)
moving file |D:/E_filing/ab/0000546960 (2)\Scan2.pdf| to destination |D:/E_filing/ab/0000546960/|
in directory  D:/E_filing/ab/0000546960 (3)
moving file |D:/E_filing/ab/0000546960 (3)\Scan3.pdf| to destination |D:/E_filing/ab/0000546960/|
in directory  D:/E_filing/ab/0001127134 (2)
moving file |D:/E_filing/ab/0001127134 (2)\Scan2.pdf| to destination |D:/E_filing/ab/0001127134/|
in directory  D:/E_filing/ab/0001127134 (3)
moving file |D:/E_filing/ab/0001127134 (3)\Scan3.pdf| to destination |D:/E_filing/ab/0001127134/|
in directory  D:/E_filing/ab/0002208885 (2)
moving file |D:/E_filing/ab/0002208885 (2)\Scan2.pdf| to destination |D:/E_filing/ab/0002208885/|
in directory  D:/E_filing/ab/0002208885 (3)
moving file |D:/E_filing/ab/0002208885 (3)\Scan3.pdf| to destination |D:/E_filing/ab/0002208885/|
in directory  D:/E_filing/ab/0002208885 (4)
moving file |D:/E_filing/ab/0002208885 (4)\Scan4.pdf| to destination |D:/E_filing/ab/0002208885/|
in dir

In [77]:
# remove empty directories
entries_ab = os.scandir(dest)

for entry in entries_ab:
    # delete directory
    try:
        os.rmdir(entry.path)
        print("deleted dir {}".format(entry.path))
    except OSError as e:
        pass

deleted dir D:/E_filing/ab/0000546960 (2)
deleted dir D:/E_filing/ab/0000546960 (3)
deleted dir D:/E_filing/ab/0001127134 (2)
deleted dir D:/E_filing/ab/0001127134 (3)
deleted dir D:/E_filing/ab/0002208885 (2)
deleted dir D:/E_filing/ab/0002208885 (3)
deleted dir D:/E_filing/ab/0002208885 (4)
deleted dir D:/E_filing/ab/0002208885 (5)
deleted dir D:/E_filing/ab/0002797415 (2)
deleted dir D:/E_filing/ab/0003883372 (2)
deleted dir D:/E_filing/ab/0003883372 (3)
deleted dir D:/E_filing/ab/0003883372(4)
deleted dir D:/E_filing/ab/0006573774(2)
deleted dir D:/E_filing/ab/0007682928 (2)
deleted dir D:/E_filing/ab/0013221392(2)
deleted dir D:/E_filing/ab/0028087451 (2)
deleted dir D:/E_filing/ab/0036802307(2)
deleted dir D:/E_filing/ab/0037830376 (2)
deleted dir D:/E_filing/ab/0043763841 (2)
deleted dir D:/E_filing/ab/0043763841 (3)
deleted dir D:/E_filing/ab/0046198146 (2)
deleted dir D:/E_filing/ab/0046198146 (3)
deleted dir D:/E_filing/ab/0047276300(2)
deleted dir D:/E_filing/ab/0047276300(3

# DONE !!!!
### now move all directories back to their original directory so that merging begins

In [80]:
dest = "D:/E_filing/ab/"
dest_old = "D:/E_filing/oct3_23/"
# scan dir for updated directory list
entries_ab = os.scandir(dest)
for entry in entries_ab:
    # start moving directories to original location
    old_destination = dest_old + entry.name
    shutil.move(entry.path, old_destination)
    print("move from {} to {}".format(entry.path, old_destination))

move from D:/E_filing/ab/0000546960 to D:/E_filing/oct3_23/0000546960
move from D:/E_filing/ab/0001127134 to D:/E_filing/oct3_23/0001127134
move from D:/E_filing/ab/0002208885 to D:/E_filing/oct3_23/0002208885
move from D:/E_filing/ab/0002797415 to D:/E_filing/oct3_23/0002797415
move from D:/E_filing/ab/0003883372 to D:/E_filing/oct3_23/0003883372
move from D:/E_filing/ab/0006573774 to D:/E_filing/oct3_23/0006573774
move from D:/E_filing/ab/0007682928 to D:/E_filing/oct3_23/0007682928
move from D:/E_filing/ab/0013221392 to D:/E_filing/oct3_23/0013221392
move from D:/E_filing/ab/0028087451 to D:/E_filing/oct3_23/0028087451
move from D:/E_filing/ab/0030260750 to D:/E_filing/oct3_23/0030260750
move from D:/E_filing/ab/0036802307 to D:/E_filing/oct3_23/0036802307
move from D:/E_filing/ab/0037830376 to D:/E_filing/oct3_23/0037830376
move from D:/E_filing/ab/0043763841 to D:/E_filing/oct3_23/0043763841
move from D:/E_filing/ab/0046198146 to D:/E_filing/oct3_23/0046198146
move from D:/E_filin

# now the fun part begins!!